In [ ]:
import polars as pl
import os
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
cfg = pl.Config()
cfg.set_tbl_rows(100)
cfg.set_fmt_str_lengths(200)

polars.config.Config

In [ ]:
uri = os.environ.get("DB_URI")
# direction 0 is southbound (to brooklyn, not bronx/manhattan)
query = """
SELECT st.trip_id,
    st.stop_id,
    st.arrival,
    st.departure,
    t.mta_id,
    t.created_at,
    t.updated_at,
    t.route_id,
    t.express,
    t.assigned
FROM stop_time st
LEFT JOIN trip t ON st.trip_id = t.id
WHERE (t.route_id = '4' OR t.route_id = '3') AND t.direction = 0
--LIMIT 1000
"""
df = pl.read_database_uri(query=query, uri=uri)
df.write_parquet("4_and_3_southbound.parquet")

In [17]:
df = df.sort(by=["arrival", "trip_id"])

In [23]:
# convert from UTC to EST
df = df.with_columns(
    arrival=pl.col("arrival").dt.convert_time_zone("America/New_York"),
    departure=pl.col("departure").dt.convert_time_zone("America/New_York"),
)

In [ ]:
# 239 = franklin ave, 250 = utica ave

# Filter data for relevant stops and routes
filtered_df = df.filter(
    ((pl.col("stop_id") == 239) | (pl.col("stop_id") == 250))
    & ((pl.col("route_id") == "3") | (pl.col("route_id") == "4"))
)

# Create a pivot table to get arrival times for each trip at each stop
trip_stops = filtered_df.select(["trip_id", "route_id", "stop_id", "arrival"]).pivot(
    index=["trip_id", "route_id"], on="stop_id", values="arrival"
)

# Rename columns for clarity
trip_stops = trip_stops.rename({"239": "arrival_239", "250": "arrival_250"})

# Filter trips that stop at both stations
trip_stops = trip_stops.filter(
    pl.col("arrival_239").is_not_null() & pl.col("arrival_250").is_not_null()
)

# Separate route 3 and route 4 trips
route3_trips = trip_stops.filter(pl.col("route_id") == "3").drop("route_id")
route4_trips = trip_stops.filter(pl.col("route_id") == "4").drop("route_id")

# Rename columns for the join
route3_trips = route3_trips.rename(
    {
        "trip_id": "trip_id_3",
        "arrival_239": "arrival_3_239",
        "arrival_250": "arrival_3_250",
    }
)

route4_trips = route4_trips.rename(
    {
        "trip_id": "trip_id_4",
        "arrival_239": "arrival_4_239",
        "arrival_250": "arrival_4_250",
    }
)

# Cross join to find all combinations
route3_trips = route3_trips.with_columns(pl.lit(1).alias("dummy"))
route4_trips = route4_trips.with_columns(pl.lit(1).alias("dummy"))
all_pairs = route3_trips.join(route4_trips, on="dummy")

# Find the cases where route 3 arrives at stop 239 before route 4,
# but also gets to stop 250 before route 4
result = (
    all_pairs.filter(
        (pl.col("arrival_3_239") < pl.col("arrival_4_239"))
        & (pl.col("arrival_3_250") < pl.col("arrival_4_250"))
    )
    .drop("dummy")
    .with_columns(
        [
            (
                (pl.col("arrival_4_239") - pl.col("arrival_3_239")).dt.total_seconds()
                / 60
            ).alias("time_diff_239_min"),
            (
                (pl.col("arrival_4_250") - pl.col("arrival_3_250")).dt.total_seconds()
                / 60
            ).alias("time_diff_250_min"),
        ]
    )
    .sort("time_diff_239_min")
)

In [44]:
result.filter(pl.col("time_diff_250_min") < 10).unique(["trip_id_3", "trip_id_4"])

trip_id_3,arrival_3_250,arrival_3_239,trip_id_4,arrival_4_250,arrival_4_239,time_diff_239_min,time_diff_250_min
str,"datetime[μs, America/New_York]","datetime[μs, America/New_York]",str,"datetime[μs, America/New_York]","datetime[μs, America/New_York]",f64,f64
"""0194a7b0-f628-7b50-bb02-20930dd947e7""",2025-01-27 08:42:06 EST,2025-01-27 08:36:06 EST,"""0194a7a3-6700-7cc0-8bbd-21a55e0af3d4""",2025-01-27 08:50:32 EST,2025-01-27 08:42:08 EST,6.033333,8.433333
"""0195664e-b37f-7ba3-8824-b4d21d138f8d""",2025-03-05 09:05:23 EST,2025-03-05 08:59:28 EST,"""0195663c-2e46-7d20-b9a5-0d22311bb43d""",2025-03-05 09:12:08 EST,2025-03-05 09:00:05 EST,0.616667,6.75
"""0194f550-b80c-7661-b47b-8f634e98c20a""",2025-02-11 10:36:48 EST,2025-02-11 10:27:42 EST,"""0194f555-0b5e-7140-9d18-d17e2fe64c51""",2025-02-11 10:43:46 EST,2025-02-11 10:40:28 EST,12.766667,6.966667
"""0194b214-b37f-72f3-8486-c85e59e074f5""",2025-01-29 09:21:35 EST,2025-01-29 09:16:08 EST,"""0194b1fd-c03d-71a0-8abe-18218a8dde0c""",2025-01-29 09:22:09 EST,2025-01-29 09:16:13 EST,0.083333,0.566667
"""01952919-5499-7c91-8c28-aca1486c4f0b""",2025-02-21 11:48:02 EST,2025-02-21 11:41:11 EST,"""01952910-a7f3-7a22-8704-67a3ae426bd0""",2025-02-21 11:49:25 EST,2025-02-21 11:43:37 EST,2.433333,1.383333
"""0194cd27-8029-7612-931f-e2c4e29dea4a""",2025-02-03 15:29:00 EST,2025-02-03 15:23:00 EST,"""0194cd30-afbd-7c93-aa20-e5cc477d2d12""",2025-02-03 15:34:52 EST,2025-02-03 15:31:54 EST,8.9,5.866667
"""01953d17-79a3-76f2-af54-11710c286f07""",2025-02-25 08:57:39 EST,2025-02-25 08:50:27 EST,"""01953d09-602a-7122-b363-eed9d67651a3""",2025-02-25 09:02:20 EST,2025-02-25 08:57:22 EST,6.916667,4.683333
"""0194cc06-c992-74e3-8c94-79594e475ff2""",2025-02-03 10:06:37 EST,2025-02-03 10:01:20 EST,"""0194cbfe-1ebd-7bf2-afc7-e4cbb44955eb""",2025-02-03 10:09:00 EST,2025-02-03 10:03:59 EST,2.65,2.383333
"""0194fa16-f79f-7a62-830e-e00a47d1d0de""",2025-02-12 08:52:12 EST,2025-02-12 08:44:58 EST,"""0194fa12-15fc-7941-8916-8146d79ef484""",2025-02-12 08:53:50 EST,2025-02-12 08:48:54 EST,3.933333,1.633333
